Scipy least squares does appear to work correctly for complex-valued data

In [1]:
import numpy as np
from regressioninc.testing.complex import ComplexGrid, generate_linear_grid
from regressioninc.testing.complex import add_gaussian_noise
from regressioninc.linear.models import add_intercept, OLS, WLS, M_estimate
from scipy.linalg import lstsq

np.random.seed(42)

coef = np.array([0.5 + 2j, -3 - 1j])
grid_r1 = ComplexGrid(r1=0, r2=10, nr=11, i1=-5, i2=5, ni=11)
grid_r2 = ComplexGrid(r1=-25, r2=-5, nr=11, i1=-5, i2=5, ni=11)
X, y = generate_linear_grid(coef, [grid_r1, grid_r2], intercept=20 + 20j)
X = add_intercept(X)
y = add_gaussian_noise(y, loc=(0, 0), scale=(3, 3))


Ordinary least squares

In [2]:
model = OLS()
model.fit(X, y)
print(model.coef_)

[ 0.41604037 +1.96401532j -2.949279   -0.95532859j
 21.07277407+20.94281445j]


In [3]:
coef, residues, rank, s = lstsq(X, y)
print(coef)
print(residues)

[ 0.41604037 +1.96401532j -2.949279   -0.95532859j
 21.07277407+20.94281445j]
331.1465809903866


Weighted least squares

In [4]:
weights = np.ones_like(y, dtype=float)
model = WLS()
model.fit(X, y, sample_weight=weights)
print(model.coef_)

[ 0.41604037 +1.96401532j -2.949279   -0.95532859j
 21.07277407+20.94281445j]


M estimate robust regression

In [5]:
import statsmodels.api as sm

M1 = sm.robust.norms.TukeyBiweight()
M2 = sm.robust.norms.TrimmedMean()
model = M_estimate(warm_start=True)
model.fit(X, y, M=M1)
print(model.coef_)
model.fit(X, y, M=M2)
print(model.coef_)
# try running 



[ 0.41604037 +1.96401532j -2.949279   -0.95532859j
 21.07277407+20.94281445j]
[ 0.41604037 +1.96401532j -2.949279   -0.95532859j
 21.07277407+20.94281445j]


Use Statsmodels to compare to MM estimates, which is a two phase M estimate
Statsmodels does an explicit conversion to double when taking initial parameters into M estimate, so technically it starts with a different initial

In [6]:
import statsmodels.api as sm

rlm = sm.RLM(y, X, M=sm.robust.norms.TukeyBiweight())
rlm_result = rlm.fit(maxiter=50, tol=1e-8, scale_est="mad", conv="sresid")
print(rlm_result.params)

rlm = sm.RLM(y, X, M=sm.robust.norms.TrimmedMean(), start_params=rlm_result.params)
rlm_result = rlm.fit(maxiter=50, tol=1e-8, scale_est="mad", conv="sresid")
print(rlm_result.params)

[ 0.4099553  +1.97897227j -2.94154042 -0.95310358j
 21.27637251+20.95118014j]
[ 0.39055873 +1.99466348j -2.92703813 -0.96725836j
 21.382982  +20.57036503j]


/home/ringo_dingo/.cache/pypoetry/virtualenvs/regressioninc-B1eGejHw-py3.10/lib/python3.10/site-packages/numpy/core/_asarray.py:130: ComplexWarning: Casting complex values to real discards the imaginary part
  arr = array(a, dtype=dtype, order=order, copy=False, subok=subok)
/home/ringo_dingo/.cache/pypoetry/virtualenvs/regressioninc-B1eGejHw-py3.10/lib/python3.10/site-packages/statsmodels/base/model.py:130: ValueWarning: unknown kwargs ['start_params']
  warnings.warn(msg, ValueWarning)
